# Lambda calculus in Kotlin

## Basic interface definition

Since `kotlin.FunctionN` is not able to express a recursive lambda type, whose parameter and return type are both itself, we define a new interface `Lambda` to express it.

In [3]:
fun interface Lambda {
    operator fun invoke(f: Lambda): Lambda
}

## Shorthand for defining a lambda expression with currying

If we want to write `λab.a` in Kotlin, embedded braces like `{ a -> { b -> a } }` are annoying.
Thus, we can use the power of currying.

In [4]:
fun lambda(block: (Lambda) -> Lambda) =
    Lambda { p0 -> block(p0) }

fun lambda(block: (Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1 -> block(p0, p1) } }

fun lambda(block: (Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2 -> block(p0, p1, p2) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3 -> block(p0, p1, p2, p3) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4 -> block(p0, p1, p2, p3, p4) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4, p5 -> block(p0, p1, p2, p3, p4, p5) } }

## Natural numbers

Then we can define natural numbers naturally.

In [5]:
val zero = lambda { f, x -> x }
val one = lambda { f, x -> f(x) }
val two = lambda { f, x -> f(f(x)) }